#**Imports**#

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm
import time
import pandas as pd
import re
import string
import numpy as np
from collections import Counter
import math
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

TreeTagger is used to perform lemmatization in **Italian language**.

In [48]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/italian.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper

bin/
bin/train-tree-tagger
bin/separate-punctuation
bin/tree-tagger
bin/tree-tagger-flush
cmd/
cmd/lookup.perl
COPYRIGHT
doc/
doc/sigdat95.pdf
doc/nemlap94.pdf
FILES
README
Release-Notes

TreeTagger version for PC-Linux installed.
Tagging scripts installed.
Italian parameter file installed.
Tagging scripts installed.
Path variables modified in tagging scripts.

You might want to add /content/treetagger/cmd and /content/treetagger/bin to the PATH variable so that you do not need to specify the full path to run the tagging scripts.



mkdir: cannot create directory ‘treetagger’: File exists
--2021-10-31 18:00:40--  https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
Resolving cis.lmu.de (cis.lmu.de)... 129.187.148.72, 2001:4ca0:4f01::5
Connecting to cis.lmu.de (cis.lmu.de)|129.187.148.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1889240 (1.8M) [application/x-gzip]
Saving to: ‘tree-tagger-linux-3.2.4.tar.gz.1’

     0K .......... .......... .......... .......... ..........  2%  251K 7s
    50K .......... .......... .......... .......... ..........  5%  505K 5s
   100K .......... .......... .......... .......... ..........  8% 34.3M 3s
   150K .......... .......... .......... .......... .......... 10% 28.8M 2s
   200K .......... .......... .......... .......... .......... 13%  513K 3s
   250K .......... .......... .......... .......... .......... 16% 29.1M 2s
   300K .......... .......... .......... .......... .......... 18% 45.9M 2s
   350K .......... ......

In [49]:
import treetaggerwrapper 

#**Load Data**#

Downloading of the `kijiji` announcements: Homework 1 Exercise 6.

In [50]:
url = 'https://www.kijiji.it/offerte-di-lavoro/offerta/informatica-e-web/'

In [51]:
def get_data_from_url(url, num_pages):

  new_url = url

  with open('/tmp/output.tsv', 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['Title', 'Description', 'Location', 'Publication Date', 'URL'])

    for i in tqdm(range(1, num_pages+1)):
      if i > 1:
        new_url = url + "?p=" + str(i)

      r = requests.get(new_url)
      soup = BeautifulSoup(r.content)
      g_data = soup.find_all("div", {"class": "item-content"})
      for item in g_data:
        tsv_writer.writerow([item.contents[1].find_all("a", {"class": "cta"})[0].text.strip(),
                             item.contents[3].text, 
                             item.contents[7].text, item.contents[9].text,
                             item.contents[1].find_all("a", {"class": "cta"})[0].get('href')])
      time.sleep(0.75)

#152 is the number of pages with job announcements on the kijiji web page in the Informatica/Grafica/Web sector     
get_data_from_url(url, 152)

100%|██████████| 152/152 [04:45<00:00,  1.88s/it]


In [52]:
tsv_file = open("/tmp/output.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

for i, row in enumerate(read_tsv):
  print(row)
  if i == 5:
    break

['Title', 'Description', 'Location', 'Publication Date', 'URL']
['Editor Premiere - videografico-a After Effects', 'Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.', 'Novara', '31 agosto, 19:02', 'https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videografico-a-after-effects/159299183']
['Ricerca nuovi collaboratori', 'Stiamo selezionando nuovi collaboratori per un\'opportunità lavorativa molto profittevole nel mondo digitale\n\nSiamo una community di oltre 14.000 persone che stanno cambiando radicalmente la propria vita sociale personale?\n\nGrazie al "DIRITTO DI LIBERO SCAMBIO" che sta RIVOLUZIONANDO il modello economico, potrai v

We **drop the duplicates** in order to not propose to users copies of the same announcement published several times, but alternative announcements instead.

In [135]:
df = pd.read_csv('/tmp/output.tsv', delimiter = '\t')
df = df.drop_duplicates()
df.count()

Title               2847
Description         2847
Location            2847
Publication Date    2847
URL                 2847
dtype: int64

In [137]:
df.head()

,Title,Description,Location,Publication Date,URL
0,Editor Premiere - videografico-a After Effects,"Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.",Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videografico-a-after-effects/159299183
1,Ricerca nuovi collaboratori,"Stiamo selezionando nuovi collaboratori per un'opportunità lavorativa molto profittevole nel mondo digitale\n\nSiamo una community di oltre 14.000 persone che stanno cambiando radicalmente la propria vita sociale personale?\n\nGrazie al ""DIRITTO DI LIBERO SCAMBIO"" che sta RIVOLUZIONANDO il modello economico, potrai veramente ottenere ciò che cerchi e desideri in modo semplice e concreto. \n\nCandidandoti potrai accedere a molti esclusivi vantaggi ed informazioni riservate SOLO ai membri di questa PRESTIGIOSA community di cui tutti vogliono fare parte.\n\nScopri SUBITO come usufruire dei vantaggi ed opportunità grazie al ""DIRITTO DI LIBERO SCAMBIO"".\nPotrai così realizzare concretamente il sogno di iniziare una nuova professione profittevole.\n\n\n\n\n",Milano,"Oggi, 18:55",https://www.kijiji.it/annunci/offerta/milano-annunci-milano/ricerca-nuovi-collaboratori/167630863
2,Tecnico Informatico,"Tecnico informatico o simile, munito di auto, per facile lavoro di installazione terminali pc da gioco sisal con superenalotto, in Milano e provincia. Possibilità di ulteriori incarichi.\nPregasi allegare CV aggiornato.\n",Milano,"Oggi, 18:07",https://www.kijiji.it/annunci/offerta/milano-annunci-milano/tecnico-informatico/167325840
3,"Cercarsi tecnico informatico, sistemista, help desk","Ricerchiamo n. 1 Tecnico Sistemista ed Helpdesk. \nLa risorsa verrà inserita all'interno dell'ufficio IT e CED a diretto supporto del Responsabile nel mantenimento generale dell'infrastruttura IT: HW e SW; Server e client anche presso clienti esterni.\n\nPrincipali attività richieste: \no Assistenza e supporto tecnico sia HW che SW agli utenti delle varie sedi in cui l'Associazione opera \no Informazione e formazione di base verso gli operatori.\no Supporto e preparazione macchine desktop e server Windows (S.O., applicazioni e servizi di tipo Office automation)\no Configurazione di SW Antivirus e di backup\no Installazione e configurazione stampanti \n\nConoscenze specifiche:\no Tematiche sistemistiche e di networking in ambiente Windows\no Remote Desktop Services\no Conoscenza applicativa e funzionale di base su router, switch, firewall, Wi-Fi e delle reti informatiche in generale.\no Ottima Conoscenza dei sistemi operativi Windows (client e server)\no Buona Conoscenza della suite...",Città Studi / Lambrate,"Oggi, 09:11",https://www.kijiji.it/annunci/offerta/milano-annunci-citta-studi-lambrate/cercarsi-tecnico-informatico-sistemista-help-desk/166051751
4,"Operatori telefonici part-time fisso 650,00","\n\nClicca sul link sottostante ""sito web"" per inviarci la tua candidatura.",Caserta,"Oggi, 04:06",https://www.kijiji.it/annunci/offerta/caserta-annunci-caserta/operatori-telefonici-part-time-fisso-650-00/167618823


#**Preprocessing**#

The main **preprocessing function** (to be applied to the columns of the Pandas DataFrame) can be found in a cell below. It contains:

*   Lowercasing;
*   Punctuation removal;
*   Processing of the special cases;
*   Tokenization;
*   Stop-words removal;
*   Lemmatization.


### 1. Normalization

Performed **before tokenization and stop-words removal** in order to control a way of tokenization better (manual correction of tokenization).

1.1 LOWERCASING;

1.2 PROCESSING OF THE **SPECIAL CASES**:

Examples below represent motivation to perform special cases processing. They are built according to the following **scheme**:

`source sentence` $=>$ 

`output of preprocessing without processing of this special case` $=>$ 

`output of preprocessing with this pecial case handling`.

**1.2.1 Hyphen:** replaced with space.

`Editor Premiere - videografico-a After Effects` $=>$ 

`editor premiere  videograficoa after effects` $=>$ 

`editor premiere  videografico a after effects`;

**1.2.2 Slash:** replaced with space.

`Modellatore 3d cad/cam` $=>$ 

`modellatore 3d cadcam` $=>$ 

`modellatore 3d cad cam`;

**1.2.3 Plus:** replaced with the word "plus", because we are interested to keep this information. 

*Assumption:* "+" could be a part of a query.

`Call center 700 euro+bonus` $=>$ 

`call center 700 eurobonus` $=>$ 

`call center 700 euro plus bonus`;

**1.2.4 Punctuation inside numeric entities:** removed while keeping the part before delimeter.

**1.2.4.1 Comma delimeter** (mostly found in monetary entities).

*Assumption:* users query may include salary specifications.

`Operatore telefonico fisso mensile 650,00 euro` $=>$ 

`operatore telefonico fisso mensile 65000 euro` $=>$ 

`operatore telefonico fisso mensile 650 euro`;

**1.2.4.2 Dot delimeter** (mostly found in time entities).

*Assumption:* users query may include workload specifications.

`Turno 14.00/ 18.15-15.45/ 20.05 .garibaldi` $=>$ 

`turno 1400 1815 1545 2005 garibaldi` $=>$ 

`turno 14 18 15 20 garibaldi`;

**1.2.5 Hashtag:** if follows after "c" replaced with the word "sharp".

*Assumption:* we would loose relevant information otherwise (i.e. aim to satisfy users requests coming from the C# developers for which specification of C# is relevant and should not be confused with just C).

*Note:* "C++" case could be handled in the same way, but we have already implicitly considered it in a satisfactory way while handling "+" sign.

`Software developer PHP Yii2 / C# / Angular JS` $=>$

 `software developer php yii2 c angular js` $=>$
 
  `software developer php yii2 csharp angular js`;

**1.2.6 Backslash n:** replaced with space. Inclusion of a "\n" sign is a specific property of the "Description" field of the dataset, because sometimes it contains reference to the web-page instead of actual description of the position.

`\n\nClicca sul link sottostante "sito web" per inviarci la tua candidatura.` $=>$ 

`\n\nclicca sul link sottostante sito web per inviarci la tua candidatura` $=>$

 `clicca sul link sottostante sito web per inviarci la tua candidatura`.

 **1.2.7 Spaces:** all the additional spaces created in the result of the special cases processing procedure are then removed.

1.3 PUNCTUATION REMOVAL:

All the punctuation going beyond special cases processing is removed because it's auxuliary for the search engine.

1.4 OTHER:

**1.4.1 Named entities.**

**1.4.1.1 Not appreviations:** are lowercased and therefore equalized with not named entities;

**1.4.1.2 Appreviations:** are lowercased and experienced delimeters removal, and therefore joined in a single word and equalized with not named entities.

*Assumption:* typical query contains them in this form.

**1.4.2 Language specifics:** left without processing (but later lemmatized).

*Assumption:* users are mostly language representatives.

### 2. Tokenization

Is performed implicitly by the **TreeTagger**.

### 3. Lemmatization

As it suggested by the web-sources:

    Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meanings to one word. 
    Text preprocessing includes both Stemming as well as Lemmatization. Many times people find these two terms confusing. Some treat these two as the same. Actually, lemmatization is preferred over Stemming because lemmatization does morphological analysis of the words.

    Applications of lemmatization are: 
    Used in comprehensive retrieval systems like search engines.
    Used in compact indexing.

So, decision to use **lemmatization rather than stemming** has been taken, and then lemmatization has been performed within **two stages.**

3.1 FOR ITALIAN TOKENS:

As it have already been mentioned before, lemmatization of Italian tokens is performed using **tagging** with treetaggerwrapper, which converts the string into a list of three-item elements divided by "\t" delimeter: `original token\ttag\tlemma`. We derive from this triple the last item, i.e. **lemma** and replace original token with it. But thereare two special cases that arise during the tagging:

**3.1.1 Numbers:** all numerical tokens are mapped by tagger to the same lemma: `@card@`, which is not a reasonable replacement within an *assumption* that numerical tokens are relevant part of users queries. Therefore, `tag` item of a triple (which is fixed to be `NUM`) is used to keep numerical tokens as originally.

**3.1.2 Vertical delimeters:** are used by tagger in the lemma part of triple to provide an alternative lemmatization for ambiguous original tokens. Since, disambiguation is not of a focus of a current task, we simply keep only the first of the proposed alternatives.

3.2 FOR ENGLISH TOKENS: 

**WordNet Lemmatizer** provided by **NLTK** is known to work fine.

### 4. Stop-Words Removal

Is an important step of preprocessing for the search engines which are built upon term frequency counting and therefore would be **shifted** by the stop-words (which are frequent and at the same time not relevant) if they are present.

Here we delete both **Italian and English** stop-words using **NLTK** database.

In [138]:
stop_words = set.union(set(stopwords.words('italian')),set(stopwords.words('english')))
lemmatizer = WordNetLemmatizer()
tagger = treetaggerwrapper.TreeTagger(TAGLANG="it", TAGDIR='treetagger/')

def column_preprocessing(col):
  #print(col)
  col = col.apply(lambda x: x.lower()) #1.1
  col = col.apply(lambda x: re.sub('-', ' ', x)) #1.2.1
  col = col.apply(lambda x: re.sub('/', ' ', x)) #1.2.2
  col = col.apply(lambda x: re.sub('\+', ' plus ', x)) #1.2.3
  col = col.apply(lambda x: re.sub(r',\d+', '', x)) #1.2.4.1
  col = col.apply(lambda x: re.sub(r'\.\d+', '', x)) #1.2.4.2
  col = col.apply(lambda x: re.sub(r'c#', 'csharp', x)) #1.2.5
  col = col.apply(lambda x: x.translate(str.maketrans(' ', ' ', string.punctuation))) #1.3
  col = col.apply(lambda x: re.sub('\n', ' ', x)) #1.2.6
  col = col.apply(lambda x: re.sub(' +', ' ', x)) #1.2.7
  #print("\nNORMALIZATION\n", col)
  col = col.apply(lambda x: tagger.tag_text(x)) #2, 3
  #print("\nTAGGER\n", col)
  col = col.apply(lambda x: [token.split('\t')[0] if (token.split('\t')[1] == 'NUM' or token.split('\t')[1] == 'PON') 
                                                  else token.split('\t')[2].split('|')[0] for token in x]) #3.1
  col = col.apply(lambda x: [lemmatizer.lemmatize(token) for token in x]) #3.2
  col = col.apply(lambda x: [token for token in x if not token in stop_words]) #4
  #print("\nOUTPUT\n")
  return col
#example output
pd.set_option('display.max_colwidth', 150)
#print("INPUT\n")
column_preprocessing(df['Title'].head())

0                   [editor, premiere, videografico, effect]
1                            [ricerca, nuovo, collaboratore]
2                                     [tecnico, informatico]
3    [cercare, tecnico, informatico, sistemista, help, desk]
4            [operatore, telefonico, part, time, fisso, 650]
Name: Title, dtype: object

*Assumption:* users **query** consists of key words that don't include request about **Publication Date** of an annocuncement or it's **URL** (reasoning behind: this information users would rather retrieve already after being proposed with relevant announcements). Therefore, for the search-engine we are mainly interested in a similarity of a query and information which is currently stored in the first three textual fields of a DataFrame: **Title, Description, Location**. 

So, in order to prepare data to the **index** construction, we will create new DataFrame in two steps:

**Step 1** (`df_search`)**:** drop auxiliary columns, and apply preprocessing function to the kept columns.

**Step 2** (`df_search_merged`)**:** merge all information regarding announcement in one field -- represent DataFrame as a collection of Documents, where each row is adocument/announcement and has an index.

In [139]:
#Step 1
df_search = df.copy()
df_search = df_search.drop(['Publication Date', 'URL'], axis=1)
df_search = df_search.apply(column_preprocessing)
df_search.head()

,Title,Description,Location
0,"[editor, premiere, videografico, effect]","[editor, video, approfondito, documentato, esperienza, ricerca, posizione, interno, struttura, richiedere, 1, conoscenza, approfondire, premiere, ...",[novara]
1,"[ricerca, nuovo, collaboratore]","[stare, selezionare, nuovo, collaboratore, unopportunità, lavorativo, molto, profittevole, mondo, digitale, essere, community, oltre, 14, persona,...",[milano]
2,"[tecnico, informatico]","[tecnico, informatico, simile, munire, auto, facile, lavoro, installazione, terminale, pc, gioco, sisal, superenalotto, milano, provincia, possibi...",[milano]
3,"[cercare, tecnico, informatico, sistemista, help, desk]","[ricercare, n, 1, tecnico, sistemista, helpdesk, risorsa, venire, inserito, allinternare, dellufficio, ced, diretto, supporto, responsabile, mante...","[città, studio, lambrate]"
4,"[operatore, telefonico, part, time, fisso, 650]","[cliccare, link, sottostante, sito, web, inviare, candidatura]",[caserta]


In [140]:
#Step 2
df_search_merged = df_search.copy()
df_search_merged['Announcement'] = df_search_merged['Title'] + df_search_merged['Description'] + df_search_merged['Location']
df_search_merged = df_search_merged.drop(df_search_merged.columns[:-1], axis=1)
pd.set_option('display.max_colwidth', 150)
df_search_merged.head()

,Announcement
0,"[editor, premiere, videografico, effect, editor, video, approfondito, documentato, esperienza, ricerca, posizione, interno, struttura, richiedere,..."
1,"[ricerca, nuovo, collaboratore, stare, selezionare, nuovo, collaboratore, unopportunità, lavorativo, molto, profittevole, mondo, digitale, essere,..."
2,"[tecnico, informatico, tecnico, informatico, simile, munire, auto, facile, lavoro, installazione, terminale, pc, gioco, sisal, superenalotto, mila..."
3,"[cercare, tecnico, informatico, sistemista, help, desk, ricercare, n, 1, tecnico, sistemista, helpdesk, risorsa, venire, inserito, allinternare, d..."
4,"[operatore, telefonico, part, time, fisso, 650, cliccare, link, sottostante, sito, web, inviare, candidatura, caserta]"


#**Search-Engine Index**#

## 1. Inverted Index


---

1.1. First, we need to **compute TF** for each term in the document. 

---

We do it inside the DataFrame field using **lambda-function** and custom function `count` that creates `Counter()` object.

In [141]:
def count(l):

  counter = Counter()
  for elem in l:
    counter[elem] += 1
  return counter

In [142]:
df_search_merged['Announcement'] = df_search_merged['Announcement'].apply(lambda x: count(x))
df_search_merged.reset_index(level=0, inplace=True) #propagate row index to a separate column
df_search_merged.head()

,index,Announcement
0,0,"{'editor': 2, 'premiere': 2, 'videografico': 1, 'effect': 2, 'video': 1, 'approfondito': 1, 'documentato': 1, 'esperienza': 1, 'ricerca': 1, 'posi..."
1,1,"{'ricerca': 1, 'nuovo': 3, 'collaboratore': 2, 'stare': 3, 'selezionare': 1, 'unopportunità': 1, 'lavorativo': 1, 'molto': 2, 'profittevole': 2, '..."
2,2,"{'tecnico': 2, 'informatico': 2, 'simile': 1, 'munire': 1, 'auto': 1, 'facile': 1, 'lavoro': 1, 'installazione': 1, 'terminale': 1, 'pc': 1, 'gioc..."
3,3,"{'cercare': 1, 'tecnico': 3, 'informatico': 2, 'sistemista': 3, 'help': 1, 'desk': 1, 'ricercare': 1, 'n': 1, '1': 1, 'helpdesk': 1, 'risorsa': 1,..."
4,4,"{'operatore': 1, 'telefonico': 1, 'part': 1, 'time': 1, 'fisso': 1, '650': 1, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'in..."



---


1.2 **Naive Inverted Index** as was introduced during classes.


---


$\forall$ `TERM` in DataFrame: create a map 

`TERM ->  all tuples (DOCUMENT INDEX, TF`),

where `(DOCUMENT INDEX, TF`)` $-$ **(given document, term frequency of a given term in a given document)**, 

given document (represented by its index) $-$ all documents, containing given term,

given term $-$ `TERM`.

So, Inverted Index is implemented as described above map. The data structure to store this map is a dictionary: `d_indexed`.

In [143]:
d_indexed = dict()
#iterating over DataFrame
for index, counter in df_search_merged.values:
  #cast Counter() object to dictionary
  d = dict(counter)
  #iterating over each dictionary zip TF with document index and save all such tuples (index, TF of the term in document with this index) in a list
  #created list becomes a value of a new dictionary with term as a key
  for (key, value) in d.items():
    if key not in list(d_indexed.keys()):
      l = list()
      l.append(tuple((index, value)))
      d_indexed[key] = l
    else:
      d_indexed[key].append(tuple((index, value)))

#some examples from the middle because ones in the beginning are too long and not convenient for scrolling
list(d_indexed.items())[-3000:-2995]

[('internaanca', [(1283, 1)]),
 ('colloquiare', [(1283, 1)]),
 ('placement', [(1284, 1), (2834, 1)]),
 ('ddesigner', [(1284, 1)]),
 ('bresciano', [(1284, 2)])]


---

1.3 We can do better, exploiting **tf-idf** instead of just frequencies.

---



1.3.1 First, for the Inverted Index with tf-idf we need to count the number of documents $N_j$, containing term $j$, $\forall j$. And then we can compute $idf_j$  as $log(\frac{n}{N_j})$ $\forall j$, where $n$ $-$ the number of documents.

$\forall$ `TERM` in DataFrame: create a new map 

`TERM ->  IDF`,

where `IDF` $-$ **IDF of a given term**, 

given term $-$ `TERM`.

The data structure to store this map is a dictionary: `Ns`.

In [144]:
Ns = dict()
n = df_search_merged.values[-1][0]+1 #number of documents correponds to an index of the last document + 1
for (key, value) in d_indexed.items():
  Ns[key] = math.log10(n/len(value))

list(Ns.items())[:5]

[('editor', 2.776701183988411),
 ('premiere', 2.5725812013324862),
 ('videografico', 2.9985499336047674),
 ('effect', 2.776701183988411),
 ('video', 1.785475108295916)]


---

1.3.2 We rearrange `d_indexed` in such a way, so it contains **tf_idf instead of just tf**.


---



In [145]:
for (key, value) in d_indexed.items():
  for i, (index, tf) in enumerate(value):
    tf_idf = tf * Ns[key]
    d_indexed[key][i] = (index, tf_idf)

list(d_indexed.items())[-3000:-2995]

[('internaanca', [(1283, 3.4756711883244296)]),
 ('colloquiare', [(1283, 3.4756711883244296)]),
 ('placement', [(1284, 3.1746411926604483), (2834, 3.1746411926604483)]),
 ('ddesigner', [(1284, 3.4756711883244296)]),
 ('bresciano', [(1284, 6.951342376648859)])]



---

1.4 Finally, we place created Inverted Index to the DataFrame `df_index` together with the **IDF** information (which will be needed to compute the **cosine similarity** with the query). 

---





Then we **save** it in a file as suggested in the assignment.

In [146]:
df_index = pd.DataFrame(columns = ['Term', 'IDF', 'Docs'])
for (key, value) in tqdm(d_indexed.items()):
  row = pd.DataFrame([[key, Ns[key], value]], columns=['Term', 'IDF', 'Docs'])
  df_index = df_index.append(row, ignore_index=True)

100%|██████████| 9900/9900 [00:26<00:00, 380.27it/s]


In [147]:
pd.set_option('display.max_colwidth', 100)
df_index

,Term,IDF,Docs
0,editor,2.776701,"[(0, 5.553402367976822), (85, 2.776701183988411), (389, 5.553402367976822), (1467, 2.77670118398..."
1,premiere,2.572581,"[(0, 5.1451624026649725), (379, 2.5725812013324862), (389, 5.1451624026649725), (555, 2.57258120..."
2,videografico,2.998550,"[(0, 2.9985499336047674), (1363, 8.995649800814302), (1860, 2.9985499336047674)]"
3,effect,2.776701,"[(0, 5.553402367976822), (389, 2.776701183988411), (555, 2.776701183988411), (652, 2.77670118398..."
4,video,1.785475,"[(0, 1.785475108295916), (35, 1.785475108295916), (85, 10.712850649775495), (86, 7.1419004331836..."
...,...,...,...
9895,rubicone,3.475671,"[(2988, 6.951342376648859)]"
9896,savignano,3.475671,"[(2988, 6.951342376648859)]"
9897,rubicona,3.475671,"[(2988, 3.4756711883244296)]"
9898,reggiano,3.475671,"[(2989, 3.4756711883244296)]"


In [148]:
df_index.to_csv('/tmp/output_index.csv', index=False)

*NOTE 1:* 

Keep the structure of such a form seems to be not convenient in a sense of organizing search. One of the **alternative** options might be to keep the document index as a column name, and tf-idf as a value on the intersection term and document index. 

Then, to perform a search, one just need to derive a cell of a table, but: 

1) That would be a complete search table and **not** what is called **"inverted index"**; 

2) This will be **time** consuming on the stage of loading data to the table; 

3) Neither it's optimal from point of view of **memory** consumption.

*NOTE 2:* 

**One more alternative** would be to do as we have done during Spark Lab, i.e. don't keep term as a unique row, but **keep as many rows as documents**, containing this term, each with it's own index. Then, to derive information regarding each document (e.d. document norm) would be easier, but that's not exactly inverted index, discussed during classes.


*NEVERTHELESS:*

We still can organized search in a quite effective way, as it's shown in the following chapter. Indeed, the slowes part of the framework is a computation of documents norms, because in such settings information about particular document is not easily available. But all documents norms do not depend on a query and could be precomputed once.

## 2. Query

In the **query processing** part we take a **query as a string**, and then convert it to a Pandas Series to be able to apply the same preprocessing function as to the documents (it takes as input column of a DataFrame). Then, to the preprocessed query we apply `count()` to derive **query term frequencies**.

With a goal to find the closest to the query documents, we need to compute **cosine similarity** between each document and query using the following formula:

\begin{align}
cos(\mathbf{q}, \mathbf{d_i}) = \frac{\sum_{j=1}^m tf-idf_{qj} \cdot tf-idf_{ij}}{||\mathbf{q}||_2 \cdot ||\mathbf{d_i}||_2} \forall i \in \{1,...,n\},
\end{align}

where $\mathbf{q}$ is a query vector, represented by its $tf-idf_{qj}$ $\forall j \in \{1,...,m\}$, and $||\mathbf{q}||_2$ $-$ its norm, 

$\mathbf{d_i}$ is a document vector, represented by its $tf-idf_{ij}$ $\forall j \in \{1,...,m\}$, and $||\mathbf{d_i}||_2$ $-$ its norm,

$m$ $-$ number of terms, $n$ $-$ number of documents.

Cycle in a cell below computes **numerator** part of the formula for each document and saves in a dictionary `docs_tf_idf`, 

where key $-$ DOCUMENT INDEX, value $-$ NUMERATOR, computed for the document with this index.

In [149]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="it", TAGDIR='treetagger/')

q = 'Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.'
q = pd.Series(q)
preprocessed_q = column_preprocessing(q)
q_tf = count(preprocessed_q[0])

print("QUERY\n", dict(q_tf))

docs_tf_idf = dict()
#iterating over only terms containing in query, because for other terms query tf-idf is equal to 0
for (q_key, q_tf) in dict(q_tf).items():
  #we have query tf, now to compute tf-idf we need also to get idf fpt each query term, which is stored in index table
  idf = df_index[df_index['Term']==q_key]['IDF'].values
  #iterating over all the documents, containing given query term, we get their tf-idf, multiply by query tf-idf of a corresponding term and save
  for (doc_id, doc_tf_idf) in list(df_index[df_index['Term']==q_key]['Docs'])[0]:
    if doc_id not in docs_tf_idf.keys():
      docs_tf_idf[doc_id] = float(doc_tf_idf*q_tf*idf)
    else:
      docs_tf_idf[doc_id] += float(doc_tf_idf*q_tf*idf)

print("\nDICTIONARY 10 LAST ITEMS")
list(docs_tf_idf.items())[-10:]

QUERY
 {'editor': 1, 'video': 1, 'approfondito': 1, 'documentato': 1, 'esperienza': 1, 'ricerca': 1, 'posizione': 1, 'interno': 1, 'struttura': 1, 'richiedere': 1, '1': 1, 'conoscenza': 2, 'approfondire': 1, 'premiere': 1, 'effect': 1, 'basilare': 1, 'davinci': 1, 'residenza': 1, 'entro': 1, '15': 1, 'km': 1, 'novara': 1, 'interessato': 1, 'possesso': 1, 'requisito': 1, 'inviare': 1, 'curriculum': 1, 'job': 1, 'giuseppegallianoit': 1, 'link': 1, 'proprio': 1, 'show': 1, 'reel': 1}

DICTIONARY 10 LAST ITEMS


[(2944, 0.06740641063712674),
 (1044, 4.563291268203765),
 (2906, 1.7843405395603822),
 (2851, 3.77988358771334),
 (390, 0.047520779521047905),
 (642, 0.047520779521047905),
 (1052, 0.9263169095477944),
 (1812, 0.9263169095477944),
 (2388, 1.8526338190955889),
 (2870, 0.9263169095477944)]

Now, we need also **norms of documents vectors** to complete the formula. For each document we are interested in, we are checking all the rows of the index. 

    If the document was contained in the row: we infer its tf-idf,

    else: we fill the corresponding value with 0.

Then we clean out all the zeros and compute Euclidian **norm** of a vector. 

Result of the computations is saved again in the dictionary, which is called `docs_norms` and has a form: 

key $-$ DOCUMENT INDEX, value $-$ NORM of a document computed for the document with this index.

In [150]:
docs_norms = dict()
for doc_id in tqdm(list(docs_tf_idf.keys())):
  doc_coords = df_index['Docs'].apply(lambda x: sum([tf_idf if index==doc_id else 0 for (index, tf_idf) in x]))
  doc_coords = doc_coords[doc_coords>0]
  doc_coords = doc_coords.apply(lambda x: x**2)
  docs_norms[doc_id] = math.sqrt(doc_coords.sum())

100%|██████████| 2763/2763 [01:54<00:00, 24.10it/s]


Finally, we can **join** two dictionaries corresponding to the **numerator** **and denominator** parts of the formula of cosine similarities.


In [151]:
similarities = dict()
for (doc_id, doc_tf_idf) in docs_tf_idf.items():
  similarities[doc_id] = doc_tf_idf/docs_norms[doc_id]

**Sorting similarities** by the value in descinding order and printing **top-5** items, we retrieve indices of the document, closes to the query.

*NOTE:* 

In fact, not cosine similarities, but cosine similarities, multiplied by the norm of the query, have been computed. We never compute norm of a query, since it's the same for all documents and doesn't influence on the result of comparison. That's why final values of similarities are greater than 1.

In [152]:
sorted_tuples = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
top_5 = sorted_tuples[:5]
print("TOP-5\n", top_5)

TOP-5
 [(0, 10.869276542141431), (389, 10.423592452961877), (463, 1.5112248886569895), (2559, 1.3331854724491985), (555, 1.2310618249902723)]


Now, we can find **top-5 announcements** correponding to found indices in the merged DataFrame and analyze them.

In [159]:
pd.set_option('display.max_colwidth', 1000)
for (index, similarity) in top_5:
  print(df_search_merged[df_search_merged.index==index]['Announcement'])

0    {'editor': 2, 'premiere': 2, 'videografico': 1, 'effect': 2, 'video': 1, 'approfondito': 1, 'documentato': 1, 'esperienza': 1, 'ricerca': 1, 'posizione': 1, 'interno': 1, 'struttura': 1, 'richiedere': 1, '1': 1, 'conoscenza': 2, 'approfondire': 1, 'basilare': 1, 'davinci': 1, 'residenza': 1, 'entro': 1, '15': 1, 'km': 1, 'novara': 2, 'interessato': 1, 'possesso': 1, 'requisito': 1, 'inviare': 1, 'curriculum': 1, 'job': 1, 'giuseppegallianoit': 1, 'link': 1, 'proprio': 1, 'show': 1, 'reel': 1}
Name: Announcement, dtype: object
389    {'copywriter': 2, 'esperto': 2, 'settore': 2, 'esperienza': 4, 'gastronomico': 2, 'cercare': 1, 'blog': 1, 'inerente': 2, 'redazione': 1, 'settimanale': 2, 'articolo': 1, 'nonché': 1, 'newsletter': 1, 'sempre': 1, 'gusto': 1, 'richiedere': 1, 'pregresso': 1, 'milano': 1}
Name: Announcement, dtype: object
463    {'lavoro': 1, 'part': 1, 'time': 1, 'giovane': 1, 'apprendista': 1, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'inviare':

Or to check up what were initial whole announcements.

In [154]:
for (index, similarity) in top_5:
  print(df[df.index==index].values)

[['Editor Premiere - videografico-a After Effects'
  'Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.'
  'Novara' '31 agosto, 19:02'
  'https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videografico-a-after-effects/159299183']]
[['Video Editor Premiere'
  'Editor video con con documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere 2) discreta conoscenza di Davinci 3) basilare conoscenza di After Effects. Indispensabile residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a propri

## 3. Screenshots

Let's collect all the stages of the search by the query in a single function to call it with different queries.

In [160]:
tagger = treetaggerwrapper.TreeTagger(TAGLANG="it", TAGDIR='treetagger/')

def search(q):

  q = pd.Series(q)
  preprocessed_q = column_preprocessing(q)
  q_tf = count(preprocessed_q[0])

  docs_tf_idf = dict()
  #iterating over only terms containing in query, because for other terms query tf-idf is equal to 0
  for (q_key, q_tf) in dict(q_tf).items():
    #we have query tf, now to compute tf-idf we need also to get idf fpt each query term, which is stored in index table
    idf = df_index[df_index['Term']==q_key]['IDF'].values
    #iterating over all the documents, containing given query term, we get their tf-idf, multiply by query tf-idf of a corresponding term and save
    for (doc_id, doc_tf_idf) in list(df_index[df_index['Term']==q_key]['Docs'])[0]:
      if doc_id not in docs_tf_idf.keys():
        docs_tf_idf[doc_id] = float(doc_tf_idf*q_tf*idf)
      else:
        docs_tf_idf[doc_id] += float(doc_tf_idf*q_tf*idf)

  docs_norms = dict()
  for doc_id in tqdm(list(docs_tf_idf.keys())):
    doc_coords = df_index['Docs'].apply(lambda x: sum([tf_idf if index==doc_id else 0 for (index, tf_idf) in x]))
    doc_coords = doc_coords[doc_coords>0]
    doc_coords = doc_coords.apply(lambda x: x**2)
    docs_norms[doc_id] = math.sqrt(doc_coords.sum())

  similarities = dict()
  for (doc_id, doc_tf_idf) in docs_tf_idf.items():
    similarities[doc_id] = doc_tf_idf/docs_norms[doc_id]

  sorted_tuples = sorted(similarities.items(), key=lambda item: item[1], reverse=True)
  top_5 = sorted_tuples[:5]
  print("TOP-5\n", top_5)

  pd.set_option('display.max_colwidth', 1000)
  for (index, similarity) in top_5:
    print(df_search_merged[df_search_merged.index==index]['Announcement'])

  for (index, similarity) in top_5:
    print(df[df.index==index].values)

### 3.1 Query 1
**Idea:** 

Simple test if we take as a query actual description of one of the announcement, we expect to get in the result index of this annoucement. Let's take the part of the content of the Description field of the announcement with index 0.

**Result:** 

Indeed, announcement with index 0 is recommended as top-1 annoucement. Although the following in top-2 announcement (index 389) has close value of cosine similarity (up to query norm), this is correct, because the contents are similar as well.

In [161]:
q1 = 'Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.'
search(q1)

100%|██████████| 2763/2763 [01:49<00:00, 25.28it/s]

TOP-5
 [(0, 10.869276542141431), (389, 10.423592452961877), (463, 1.5112248886569895), (2559, 1.3331854724491985), (555, 1.2310618249902723)]
0    {'editor': 2, 'premiere': 2, 'videografico': 1, 'effect': 2, 'video': 1, 'approfondito': 1, 'documentato': 1, 'esperienza': 1, 'ricerca': 1, 'posizione': 1, 'interno': 1, 'struttura': 1, 'richiedere': 1, '1': 1, 'conoscenza': 2, 'approfondire': 1, 'basilare': 1, 'davinci': 1, 'residenza': 1, 'entro': 1, '15': 1, 'km': 1, 'novara': 2, 'interessato': 1, 'possesso': 1, 'requisito': 1, 'inviare': 1, 'curriculum': 1, 'job': 1, 'giuseppegallianoit': 1, 'link': 1, 'proprio': 1, 'show': 1, 'reel': 1}
Name: Announcement, dtype: object
389    {'copywriter': 2, 'esperto': 2, 'settore': 2, 'esperienza': 4, 'gastronomico': 2, 'cercare': 1, 'blog': 1, 'inerente': 2, 'redazione': 1, 'settimanale': 2, 'articolo': 1, 'nonché': 1, 'newsletter': 1, 'sempre': 1, 'gusto': 1, 'richiedere': 1, 'pregresso': 1, 'milano': 1}
Name: Announcement, dtype: object
463    {

In [163]:
df[df.index==0]

,Title,Description,Location,Publication Date,URL
0,Editor Premiere - videografico-a After Effects,"Editor video con con approfondita e documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere e After Effects, basilare conoscenza di Davinci, residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.",Novara,"31 agosto, 19:02",https://www.kijiji.it/annunci/offerta/messina-annunci-novara-di-sicilia/editor-premiere-videografico-a-after-effects/159299183


In [164]:
df[df.index==389]

,Title,Description,Location,Publication Date,URL
389,Video Editor Premiere,Editor video con con documentata esperienza si ricerca per posizione interna alla struttura; si richiede 1) conoscenza approfondita di Premiere 2) discreta conoscenza di Davinci 3) basilare conoscenza di After Effects. Indispensabile residenza entro 15 km da Novara\nSe interessati e in possesso di tutti i requisiti inviare un curriculum a Job AT giuseppegalliano.it e link a proprio show reel.,Novara,"25 ottobre, 12:43",https://www.kijiji.it/annunci/offerta/novara-annunci-novara/video-editor-premiere/167407426


### 3.2 Query 2
**Idea:** 

Very short query with the name of specialization.

**Result:** 

Indeed, the search seems to be Title-oriented: announcements with similar to query Titles are selected, meanwhile Description part of announcements is chosen as the shortest possible: no description, reference to web-page.

In [165]:
q2 = 'software developer'
search(q2)

100%|██████████| 502/502 [00:19<00:00, 25.15it/s]

TOP-5
 [(1286, 0.5242870506607974), (2791, 0.39704199105321714), (2240, 0.3675930210682385), (1634, 0.35021954493257523), (2624, 0.3303452821462126)]
1286    {'poartime': 1, 'disponibilita': 1, 'immmediare': 1, '6h': 1, 'gg': 1, '800': 1, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'inviare': 1, 'candidatura': 1, 'caserta': 1}
Name: Announcement, dtype: object
2791    {'team': 1, 'leader': 1, 'esperienza': 1, 'melito': 1, 'napoli': 2, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'inviare': 1, 'candidatura': 1}
Name: Announcement, dtype: object
2240    {'centralino': 2, 'receptionist': 1, 'risorsa': 1, 'svolgere': 1, 'mansione': 1, 'reception': 1, 'essere': 3, 'inoltre': 1, 'supporto': 1, 'allufficiare': 1, 'fornitorisi': 1, 'valutare': 1, 'candidatura': 2, 'preferibilmente': 1, 'possesso': 1, 'seguente': 1, 'requisito': 1, 'età': 1, 'comprendere': 1, '18': 1, '29': 1, 'anno': 1, 'minimo': 1, 'esperienza': 1, 'lavorativo': 1, 'conoscenza': 4, 'mo

### 3.3 Query 3
**Idea:** 

Very long query with the content, which does not correspond to any particular announcement (description of who is Java-developer in Italian).

**Result:** 

Top-1 search result are courses of programming, all other candidates in top-5 also include the general content about the programming, which reflects the nature of a query correctly.

In [167]:
q3 = 'Diventare programmatore informatico richiede una formazione tecnica molto specifica, ad esempio una laurea in Informatica, Ingegneria Informatica, Software Engineering. Esistono però anche corsi online e offline per studiare programmazione da autodidatta, imparando i vari linguaggi di programmazione in autonomia e preparando un portfolio con i progetti realizzati per dimostrare le proprie abilità nel coding.'
search(q3)

100%|██████████| 1314/1314 [00:52<00:00, 25.12it/s]

TOP-5
 [(1807, 2.110456119644632), (2873, 1.5337882675587495), (2928, 1.3589919610105483), (922, 1.223458862642477), (1094, 1.2187270994087147)]
1807    {'disegnatore': 1, 'junior': 2, 'importante': 1, 'società': 1, 'consulenza': 1, 'ingegneristica': 1, 'operante': 1, '36': 1, 'paese': 1, 'mondo': 1, 'stare': 1, 'selezionare': 1, 'undisegnatore': 1, 'riportare': 1, 'responsabile': 1, 'team': 1, 'progetto': 1, 'internazionale': 1, 'occupare': 1, 'redigere': 1, 'grafico': 1, 'struttura': 1, 'civile': 1, 'lavorare': 2, 'contesto': 1, 'sereno': 1, 'innovativo': 1, 'dinamico': 1, 'allinterno': 1, 'giovane': 1, 'affiatare': 1, 'grupporequisitidiploma': 1, 'laurea': 1, 'tecnicaconoscenza': 1, 'lingua': 1, 'inglese': 1, 'b1eccellente': 1, 'conoscenza': 1, 'autocaddisponibilità': 1, 'genova': 2, 'fuori': 1, 'sedecompetenza': 1, 'trasversalicapacità': 1, 'agire': 1, 'proattivamenteprecisione': 1, 'organizzazionecapacità': 1, 'analisi': 1, 'spirito': 1, 'criticocapacità': 1, 'gestire': 1, 'forte'

### 3.4 Query 4
**Idea:** 

Short query with salary and location specifications.

**Result:** 

Looks like preference in search is given to the number, meanwhile location information is ignored.

In [169]:
q4 = 'intelligenza artificiale engineer 1000 euro milano'
search(q4)

100%|██████████| 544/544 [00:21<00:00, 25.05it/s]

TOP-5
 [(522, 1.858215015601881), (761, 1.6233151494856204), (2714, 1.586982468774548), (1668, 1.0486018546113232), (593, 1.030570830840662)]
522    {'progettista': 1, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'inviare': 1, 'candidatura': 1, 'napoli': 1}
Name: Announcement, dtype: object
761    {'corso': 1, 'certificazione': 1, 'ecdl': 1, 'cliccare': 1, 'link': 1, 'sottostante': 1, 'sito': 1, 'web': 1, 'inviare': 1, 'candidatura': 1, 'treviso': 1}
Name: Announcement, dtype: object
2714    {'lazio': 1, 'visioture': 1, 'ricercare': 1, 'angular': 2, 'developer': 1, '2': 1, '3': 1, 'anno': 1, 'esperienza': 2, 'ruolo': 1, 'requisire': 1, 'conoscenza': 6, '6': 1, 'javascript': 1, 'typescript': 1, 'html5': 1, 'css3': 1, 'base': 3, 'apache': 1, 'web': 1, 'server': 1, 'database': 1, 'sql': 1, 'vuejs': 1, 'react': 1, 'lingua': 1, 'inglese': 1, 'capacità': 2, 'lavoro': 2, 'team': 1, 'passione': 1, 'mondo': 1, 'digital': 2, 'sede': 1, 'rom': 2, 'offrire': 1, 'contratto': 1, 